In [1]:
!pip install geopy requests --quiet
print("Packages checked/installed.")


Packages checked/installed.


In [2]:
import json, os

DATA_PATH = r"C:\Users\barya\metro_project\.ipynb_checkpoints\stations.json"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"stations.json not found at {DATA_PATH}. Make sure you saved it there.")

with open(DATA_PATH, "r") as f:
    raw = json.load(f)

stations = {}
for name, info in raw.items():
    stations[name] = {
        "coords": (info["lat"], info["lon"]),
        "lines": set(info.get("lines", []))
    }

print("Loaded", len(stations), "stations. Example:", list(stations.items())[:1])


Loaded 14 stations. Example: [('Versova', {'coords': (19.1509, 72.8236), 'lines': {'Line 1 (Blue)'}})]


In [4]:
# Cell 3
import math, heapq
from collections import defaultdict

def haversine_km(a, b):
    lat1, lon1 = a; lat2, lon2 = b
    R = 6371.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    x = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2 * R * math.asin(math.sqrt(x))

def dijkstra(graph, source, target):
    pq = [(0, source)]; dist = {source: 0}; prev = {}
    while pq:
        d,u = heapq.heappop(pq)
        if u == target: break
        if d > dist.get(u, 1e18): continue
        for v,w in graph.get(u, []):
            nd = d + w
            if nd < dist.get(v, 1e18):
                dist[v] = nd; prev[v] = u
                heapq.heappush(pq, (nd, v))
    if target not in dist:
        return None, None
    path = []
    cur = target
    while cur != source:
        path.append(cur)
        cur = prev[cur]
    path.append(source)
    path.reverse()
    return path, dist[target]

def fare_from_distance_km(km):
    slabs = [(3,10),(12,20),(18,30),(24,40),(30,50),(36,60),(42,70)]
    for limit,fare in slabs:
        if km <= limit: return fare
    return 80

def find_nearest_station(latlon):
    best=None; bestd=1e18
    for name,info in stations.items():
        d = haversine_km(latlon, info["coords"])
        if d < bestd:
            best, bestd = name, d
    return best, bestd


In [5]:
# Cell 4
LINES = {
    "Line 1 (Blue)": ["Versova","Azad Nagar","D N Nagar","Andheri","Chakala (Airport Road)","Marol Naka","Saki Naka","Asalpha","Jagruti Nagar","Ghatkopar"],
    "Line 2A (Yellow)": ["Dahisar East","Borivali","Malad","Andheri West","D N Nagar"]
}

graph = defaultdict(list)
for line_name, seq in LINES.items():
    filtered = [s for s in seq if s in stations]   # ignore any missing stations
    for i in range(len(filtered)-1):
        a,b = filtered[i], filtered[i+1]
        d = haversine_km(stations[a]["coords"], stations[b]["coords"])
        graph[a].append((b,d)); graph[b].append((a,d))

print("Graph has", len(graph), "stations (nodes). Example neighbors for D N Nagar ->", graph.get("D N Nagar"))


Graph has 14 stations (nodes). Example neighbors for D N Nagar -> [('Azad Nagar', 3.0512218890264156), ('Andheri', 0.033422362675360626), ('Andheri West', 1.0611200379969912)]


In [7]:
# Cell 5 - interactive route finder
import requests
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

def geocode_address(addr):
    geolocator = Nominatim(user_agent="metro-notebook")
    try:
        loc = geolocator.geocode(addr + " Mumbai", timeout=10)
        if loc: return (loc.latitude, loc.longitude)
    except GeocoderTimedOut:
        time.sleep(1)
    return None

def deduce_segments(path):
    if not path: return []
    segments = []
    current_line = None
    seg_stations = [path[0]]
    for i in range(len(path)-1):
        s,t = path[i], path[i+1]
        common = stations[s]["lines"].intersection(stations[t]["lines"])
        chosen_line = next(iter(common)) if common else "Unknown"
        if chosen_line != current_line:
            if current_line is not None:
                segments.append((current_line, seg_stations))
            current_line = chosen_line
            seg_stations = [s, t]
        else:
            seg_stations.append(t)
    segments.append((current_line, seg_stations))
    return segments

# 1) detect location by IP (quick) or manual
def get_ip_location():
    try:
        r = requests.get("https://ipinfo.io/json", timeout=5); js=r.json()
        if "loc" in js: return tuple(map(float, js["loc"].split(",")))
    except Exception:
        return None

ip_loc = get_ip_location()
if ip_loc:
    print("Detected location by IP (approx):", ip_loc)
    use = input("Use this as your current location? (y/n) [y]: ").strip().lower() or "y"
else:
    print("Could not auto-detect your location.")

if ip_loc and use == "y":
    user_loc = ip_loc
else:
    s = input("Enter your location as 'lat,lon' OR an address (e.g., 'Andheri West'): ").strip()
    if "," in s:
        user_loc = tuple(map(float, s.split(",")))
    else:
        g = geocode_address(s)
        if not g:
            print("Couldn't geocode. Please enter lat,lon format (e.g., 19.12,72.84)")
            raise SystemExit
        user_loc = g

orig, od = find_nearest_station(user_loc)
print(f"Nearest origin station: {orig} ({od*1000:.0f} m away)")

# 2) destination
dest_input = input("Enter your destination station name OR address: ").strip()
if dest_input in stations:
    dest_station = dest_input
else:
    g = geocode_address(dest_input)
    if not g:
        print("Couldn't geocode destination. Try a station name from the list.")
        raise SystemExit
    dest_station, dd = find_nearest_station(g)
    print(f"Nearest metro station to destination: {dest_station} ({dd*1000:.0f} m away)")

# 3) path
if orig not in graph or dest_station not in graph:
    print("Either origin or destination not in built graph. Make sure stations and LINES match.")
else:
    path, total_km = dijkstra(graph, orig, dest_station)
    if not path:
        print("No path found (graph incomplete).")
    else:
        total_time_min = total_km / 30.0 * 60.0
        fare = fare_from_distance_km(total_km)
        print("\n=== ROUTE ===")
        print(" -> ".join(path))
        print(f"Distance: {total_km:.2f} km, Time (approx): {total_time_min:.0f} min, Fare: ₹{fare}\n")
        segs = deduce_segments(path)
        print("Step-by-step:")
        for ln, sts in segs:
            segdist = sum(haversine_km(stations[sts[i]]["coords"], stations[sts[i+1]]["coords"]) for i in range(len(sts)-1))
            print(f" - Take {ln} from {sts[0]} -> {sts[-1]} ({len(sts)-1} stops, ~{segdist:.2f} km)")


Detected location by IP (approx): (20.2724, 85.8338)


Use this as your current location? (y/n) [y]:  N
Enter your location as 'lat,lon' OR an address (e.g., 'Andheri West'):  Marol Naka


Nearest origin station: Marol Naka (996 m away)


Enter your destination station name OR address:  Malad



=== ROUTE ===
Marol Naka -> Chakala (Airport Road) -> Andheri -> D N Nagar -> Andheri West -> Malad
Distance: 8.79 km, Time (approx): 18 min, Fare: ₹20

Step-by-step:
 - Take Line 1 (Blue) from Marol Naka -> D N Nagar (3 stops, ~3.04 km)
 - Take Line 2A (Yellow) from D N Nagar -> Malad (2 stops, ~5.75 km)


In [8]:
import os
out_dir = "../outputs"
os.makedirs(out_dir, exist_ok=True)  # Create folder if it doesn't exist

out_path = os.path.join(out_dir, "last_route.json")
with open(out_path, "w") as f:
    json.dump(out, f, indent=2)

print("Saved to", out_path)


Saved to ../outputs\last_route.json
